In [1]:
path = '/Users/kaylakim/Documents/portfolio/Forecasting-Sticker-Sales/'

In [2]:
import pandas as pd

In [3]:
train = pd.read_csv(path + 'train.csv', index_col=0)
train.head()

,date,country,store,product,num_sold
id,,,,,
0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


In [4]:
train.shape

(230130, 5)

In [5]:
train.isnull().sum()

date           0
country        0
store          0
product        0
num_sold    8871
dtype: int64

In [6]:
train = train.dropna()
train.isnull().sum()

date        0
country     0
store       0
product     0
num_sold    0
dtype: int64

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 221259 entries, 1 to 230129
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   date      221259 non-null  object 
 1   country   221259 non-null  object 
 2   store     221259 non-null  object 
 3   product   221259 non-null  object 
 4   num_sold  221259 non-null  float64
dtypes: float64(1), object(4)
memory usage: 10.1+ MB


In [8]:
len(train.date.unique())

2557

In [9]:
len(train.country.unique())

6

In [10]:
len(train.store.unique())

3

In [11]:
len(train['product'].unique())

5

In [12]:
train.columns

Index(['date', 'country', 'store', 'product', 'num_sold'], dtype='object')

In [13]:
train[['country', 'store', 'product']]

,country,store,product
id,,,
1,Canada,Discount Stickers,Kaggle
2,Canada,Discount Stickers,Kaggle Tiers
3,Canada,Discount Stickers,Kerneler
4,Canada,Discount Stickers,Kerneler Dark Mode
5,Canada,Stickers for Less,Holographic Goose
...,...,...,...
230125,Singapore,Premium Sticker Mart,Holographic Goose
230126,Singapore,Premium Sticker Mart,Kaggle
230127,Singapore,Premium Sticker Mart,Kaggle Tiers


In [14]:
# Import label encoder 
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in 3 columns
for value in train[train.columns[1:4]]:
    train[value] = label_encoder.fit_transform(train[value])
train.head()


,date,country,store,product,num_sold
id,,,,,
1,2010-01-01,0,0,1,973.0
2,2010-01-01,0,0,2,906.0
3,2010-01-01,0,0,3,423.0
4,2010-01-01,0,0,4,491.0
5,2010-01-01,0,2,0,300.0


In [15]:
train.num_sold = train.num_sold.map(lambda x: round(x))
train

,date,country,store,product,num_sold
id,,,,,
1,2010-01-01,0,0,1,973
2,2010-01-01,0,0,2,906
3,2010-01-01,0,0,3,423
4,2010-01-01,0,0,4,491
5,2010-01-01,0,2,0,300
...,...,...,...,...,...
230125,2016-12-31,5,1,0,466
230126,2016-12-31,5,1,1,2907
230127,2016-12-31,5,1,2,2299


In [16]:
train.describe()

,country,store,product,num_sold
count,221259.000000,221259.000000,221259.000000,221259.000000
mean,2.537524,1.011358,2.079301,752.527382
std,1.705006,0.811057,1.383619,690.165445
min,0.000000,0.000000,0.000000,5.000000
25%,1.000000,0.000000,1.000000,219.000000
50%,2.000000,1.000000,2.000000,605.000000
75%,4.000000,2.000000,3.000000,1114.000000
max,5.000000,2.000000,4.000000,5939.000000


In [17]:
y = train['num_sold']
# Drop ['date', 'num_sold'] columns
X = train.drop(['date', 'num_sold'], axis=1)
X.head()

,country,store,product
id,,,
1,0,0,1
2,0,0,2
3,0,0,3
4,0,0,4
5,0,2,0


In [18]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3)

In [19]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
y_pred = np.round(model.predict(X_valid))

In [20]:
y_pred

array([   5.,  199.,  113., ...,  812., 1696.,   14.])

In [21]:
# check train, valid score
train_score = model.score(X_train, y_train)
valid_score = model.score(X_valid, y_valid)
print(f'train_score= {train_score*100} %')
print(f'valid_score= {valid_score*100} %')

train_score= 94.03863734222196 %
valid_score= 94.16329233086009 %


In [22]:
df = pd.DataFrame({'Actual': pd.DataFrame(y_valid.values)[0].values,
                  'Prediction': pd.DataFrame(y_pred)[0].values.astype(int)})
df

,Actual,Prediction
0,6,5
1,257,199
2,84,113
3,1413,1384
4,2102,2643
...,...,...
66373,778,903
66374,499,430
66375,964,812
66376,2069,1696


In [23]:
from sklearn import metrics
print(f'mean_squared_error_score= {metrics.mean_squared_error(y_valid, y_pred)}')

mean_squared_error_score= 28226.828000241043


In [24]:
sub = pd.read_csv(path + 'sample_submission.csv')
sub

,id,num_sold
0,230130,100
1,230131,100
2,230132,100
3,230133,100
4,230134,100
...,...,...
98545,328675,100
98546,328676,100
98547,328677,100
98548,328678,100


In [25]:
test = pd.read_csv(path + 'test.csv', index_col=0)
test

,date,country,store,product
id,,,,
230130,2017-01-01,Canada,Discount Stickers,Holographic Goose
230131,2017-01-01,Canada,Discount Stickers,Kaggle
230132,2017-01-01,Canada,Discount Stickers,Kaggle Tiers
230133,2017-01-01,Canada,Discount Stickers,Kerneler
230134,2017-01-01,Canada,Discount Stickers,Kerneler Dark Mode
...,...,...,...,...
328675,2019-12-31,Singapore,Premium Sticker Mart,Holographic Goose
328676,2019-12-31,Singapore,Premium Sticker Mart,Kaggle
328677,2019-12-31,Singapore,Premium Sticker Mart,Kaggle Tiers


In [26]:
test.columns[1:]

Index(['country', 'store', 'product'], dtype='object')

In [27]:
test = test.drop('date', axis=1)

# Encode labels in 3 columns
for value in test[test.columns]:
    test[value] = label_encoder.fit_transform(test[value])
test.head()

,country,store,product
id,,,
230130,0,0,0
230131,0,0,1
230132,0,0,2
230133,0,0,3
230134,0,0,4


In [28]:
test = np.round(model.predict(test))
test

array([  98.,  716.,  594., ..., 1607.,  876., 1027.])

In [29]:
type(test[7])

numpy.float64

In [30]:
sub.num_sold = test.astype(int)
sub

,id,num_sold
0,230130,98
1,230131,716
2,230132,594
3,230133,324
4,230134,379
...,...,...
98545,328675,271
98546,328676,1935
98547,328677,1607
98548,328678,876


In [31]:
sub.to_csv('submission.csv', index=False)